Author: David Díaz

original notebook: ../remote_sensing_plots/notebooks

In [1]:
import os
import pdal
import geopandas as gpd
import numpy as np
from scipy.spatial import ConvexHull
from shapely.geometry import Polygon 

In [2]:
PLOTS = '../data/dev/features/plot_features.geojson'
plots = gpd.read_file(PLOTS).set_index('uuid')
plots[plots.columns[:10]].head()

,lat,lon,orig_id,source,meas_yr,ecoregion3,agency,plot_size_ac,bulk_dens,soil_depth
uuid,,,,,,,,,,
00027724,44.656847,-118.217206,061609823230090,USFS-BLUE-MOUNTAINS,2015,blue_mountains,USFS,0.250,106,118.0
0080963b,43.028673,-123.898747,72-04,BLM-COOS,2011,coast_range,BLM,0.125,103,113.0
0083eb5e,46.486165,-121.091325,0083eb5e-aed6-4c22-9412-bc116d072ed9,WA-DNR,2013,eastern_cascades_slopes_and_foothills,WADNR,0.100,128,113.0
008dc9d1,48.178724,-121.981598,008dc9d1-d2a0-4ee5-ba75-592318d06c1b,WA-DNR,2014,north_cascades,WADNR,0.100,97,99.0
00900e38,43.771495,-123.753809,73-25,BLM-COOS,2010,coast_range,BLM,0.125,103,113.0


In [3]:
plots.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 5089 entries, 00027724 to fff7e1c3
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   lat                  5089 non-null   float64 
 1   lon                  5089 non-null   float64 
 2   orig_id              5089 non-null   object  
 3   source               5089 non-null   object  
 4   meas_yr              5089 non-null   int64   
 5   ecoregion3           5089 non-null   object  
 6   agency               5089 non-null   object  
 7   plot_size_ac         5089 non-null   float64 
 8   bulk_dens            5089 non-null   int64   
 9   soil_depth           5089 non-null   float64 
 10  pct_clay_surf        5089 non-null   int64   
 11  pct_rock_surf        5089 non-null   int64   
 12  pct_sand_surf        5089 non-null   int64   
 13  distance_to_water_m  5089 non-null   float64 
 14  pot_veg_type         5089 non-null   int64   
 15  epsg   

In [4]:
def classify(infile, out_dir, to_epsg):
    out_base = os.path.basename(infile)
    outfile = os.path.join(out_dir, out_base)
    
    reader = pdal.Reader.las(infile)
    reproject = pdal.Filter.reprojection(out_srs=f"EPSG:{to_epsg}")
    blank = pdal.Filter.assign(assignment="Classification[:]=0")
    outlier = pdal.Filter.outlier()
    elm = pdal.Filter.elm()
    smrf = pdal.Filter.smrf()
    hag = pdal.Filter.hag_nn()
    writer = pdal.Writer.las(
        outfile,
        extra_dims="HeightAboveGround=float32",
        a_srs=f"EPSG:{to_epsg}",
    )

    pipeline = reader | blank | outlier | elm | smrf | hag | writer
    pipeline.execute()

    return outfile

In [5]:
def make_rasters(infile, out_dir, bbox, resolution=0.5):
    USEFUL = "Classification[0:6],Classification[8:17]"
    out_base = os.path.basename(infile).split(".")[0]
    reader = pdal.Reader.las(
        infile,
        extra_dims='HeightAboveGround=float32'
    )

    xmin, ymin, xmax, ymax = bbox
    bounds = f"([{xmin}, {xmax}], [{ymin}, {ymax}])"

    rng = pdal.Filter.range(limits=USEFUL)
    
    dtm = pdal.Writer.gdal(
        os.path.join(out_dir, f"{out_base}_dtm.tif"),
        resolution=resolution,
        bounds=bounds,
        dimension="Z",
        radius=resolution,
        data_type="float32",
        output_type="mean",
        where="Classification==2"
    )

    chm = pdal.Writer.gdal(
        os.path.join(out_dir, f"{out_base}_chm.tif"),
        resolution=resolution,
        bounds=bounds,
        dimension="HeightAboveGround",
        data_type="float32",
        output_type="max",
    )
    
    pipeline = reader | rng | dtm | chm
    pipeline.execute()

    assign = pdal.Filter.assign(value = [
          "Intensity = Intensity / 256"
      ])
    intensity = pdal.Writer.gdal(
        os.path.join(out_dir, f"{out_base}_intensity.tif"),
        resolution=resolution,
        bounds=bounds,
        dimension="Intensity",
        data_type="float32",
        output_type="mean"
    )
    
    if np.max(pipeline.arrays[0]['Intensity']) > 255:
        is16Bit = True
    else:
        is16Bit = False
        
    ipipeline = reader | rng
    if is16Bit:
      ipipeline |= assign
    ipipeline |= intensity
    ipipeline.execute()
    
    return True

In [6]:
def calc_crown_widths(hull):
    """
    Finds the smallest bounding rectangle from a convex hull and
    returns lengths of major and minor axes.

    Adapted from: https://stackoverflow.com/a/33619018
    """
    from scipy.ndimage import rotate
    
    pi2 = np.pi/2.

    # get points of the convex hull
    hull_points = hull.points[hull.vertices]

    # calculate edge angles
    edges = hull_points[1:] - hull_points[:-1]
    angles = np.arctan2(edges[:, 1], edges[:, 0])
    angles = np.abs(np.mod(angles, pi2))
    angles = np.unique(angles)

    # find rotation matrices
    rotations = np.vstack([
        np.cos(angles),
        np.cos(angles-pi2),
        np.cos(angles+pi2),
        np.cos(angles)]).T

    rotations = rotations.reshape((-1, 2, 2))

    # apply rotations to the hull
    rot_points = np.dot(rotations, hull_points.T)

    # find the bounding points
    min_x = np.nanmin(rot_points[:, 0], axis=1)
    max_x = np.nanmax(rot_points[:, 0], axis=1)
    min_y = np.nanmin(rot_points[:, 1], axis=1)
    max_y = np.nanmax(rot_points[:, 1], axis=1)

    # find the box with the best area
    areas = (max_x - min_x) * (max_y - min_y)
    best_idx = np.argmin(areas)

    # return the best box
    x1 = max_x[best_idx]
    x2 = min_x[best_idx]
    y1 = max_y[best_idx]
    y2 = min_y[best_idx]

    ll = np.array((x1, y1))
    ul = np.array((x1, y2))
    lr = np.array((x2, y1))

    width = np.linalg.norm(lr - ll)
    height = np.linalg.norm(ul - ll)

    return max(width, height), min(width, height)

In [7]:
def get_crowns(arr, srs):
    clusters = np.unique(arr['ClusterID'])
    clusters = clusters[clusters>0]
    geoms = []
    heights = []
    topx = []
    topy = []
    max_widths = []
    orth_widths = []
    hull_surfs = []
    hull_vols = []
    for c in clusters:
        msk = arr['ClusterID'] == c
        x = arr['X'][msk]
        y = arr['Y'][msk]
        z = arr['HeightAboveGround'][msk]
        xy = np.stack((x, y), axis=-1)
        xyz = np.stack((x, y, z), axis=-1)
        hullxy = ConvexHull(xy)
        hullxyz = ConvexHull(xyz)
        
        geoms.append(Polygon(hullxy.points[hullxy.vertices]))
        heights.append(z.max())
        topx.append(x[z.argmax()])
        topy.append(y[z.argmax()])
        max_width, orth_width = calc_crown_widths(hullxy)
        max_widths.append(max_width)
        orth_widths.append(orth_width)
        hull_surfs.append(hullxyz.area)
        hull_vols.append(hullxyz.volume)
    gdf = gpd.GeoDataFrame(
        data=np.array([clusters, topx, topy, heights, max_widths, orth_widths, hull_surfs, hull_vols]).T, 
        columns=['TAO_ID', 'TOP_X', 'TOP_Y', 'TOP_HAG', 'MCW', 'MCW90', 'HULL_AREA', 'HULL_VOLUME'], 
        geometry=geoms,
        crs=srs
    )
    gdf['TAO_ID'] = gdf['TAO_ID'].astype(int)

    pts = gpd.GeoDataFrame(
        geometry=gpd.points_from_xy(gdf.TOP_X, gdf.TOP_Y),
        crs=srs
    )
    pts = pts.to_crs(epsg=4326)
    gdf['TOP_X'] = pts.geometry.x
    gdf['TOP_Y'] = pts.geometry.y
    
    return gdf.to_crs(epsg=4326)

In [8]:
def treeseg(infile, out_dir):
    out_base = os.path.basename(infile).split('.')[0]
    outfile = os.path.join(out_dir, f"{out_base}_crowns.geojson")

    USEFUL = "Classification[0:6],Classification[8:17]"
    
    reader = pdal.Reader.las(
        infile,
        extra_dims='HeightAboveGround=float32'
    )
    rng = pdal.Filter.range(limits=USEFUL)
    sort_hag = pdal.Filter.sort(dimension="HeightAboveGround", order="DESC")
    treeseg = pdal.Filter.litree(min_height=2.0)

    writer = pdal.Writer.las(
        outfile.replace('.geojson', '.laz'),
        extra_dims="HeightAboveGround=float32,ClusterID=int32",
        a_srs=f"EPSG:{to_epsg}",
    )

    pipeline = reader | rng | sort_hag | treeseg | writer
    pipeline.execute()

    arr = pipeline.arrays[0]
    srs = pipeline.srswkt2
    crowns = get_crowns(arr, srs)
    crowns.to_file(outfile, driver='GeoJSON')
    
    return outfile

In [9]:
INFILE = "0080963b-a3ff-4081-be77-02eb7b458d30_or-south-coast_2008.laz"

In [10]:
uuid = INFILE.split('-')[0]
xmin, ymin, xmax, ymax = plots.loc[uuid, ['utm_xmin', 'utm_ymin', 'utm_xmax', 'utm_ymax']]
to_epsg = plots.loc[uuid, 'epsg']
xmin, ymin, xmax, ymax, to_epsg

(426718.4381302149, 4764330.7246319, 426838.4381302149, 4764450.7246319, 32610)

In [12]:
%%time
outlaz = classify(INFILE, out_dir="./classified", to_epsg=to_epsg)

CPU times: user 132 ms, sys: 0 ns, total: 132 ms
Wall time: 131 ms


In [13]:
%%time
make_rasters(outlaz, out_dir="./classified", bbox=(xmin, ymin, xmax, ymax), resolution=0.5)

CPU times: user 43.9 ms, sys: 0 ns, total: 43.9 ms
Wall time: 43.5 ms


True

In [14]:
%%time
treeseg(outlaz, "./classified")

CPU times: user 1.32 s, sys: 0 ns, total: 1.32 s
Wall time: 1.31 s


'./classified/0080963b-a3ff-4081-be77-02eb7b458d30_or-south-coast_2008_crowns.geojson'